# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación

### Minería de Datos: Curso académico 2020-2021

### Integrantes:

* Gonzalo Pinto Perez
* Yeremi Martin Huaman Torres

# 1. Preliminares

Para empezar vamos a cargar las librerias que utilizaremos durante el desarrollo de la práctica

In [ ]:
# Third party
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.impute import *
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn import FunctionSampler
from imblearn.pipeline import make_pipeline
from scipy.stats import shapiro
from sklearn.compose import make_column_transformer
from sklearn.metrics import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from sklearn.compose import make_column_selector, make_column_transformer




# Importamos nuestro propio fichero de utilidades
import md_grupoa_practica1extra_ficheroutilidad as utils



## *1.1 Variables globales*

Fijamos la semilla que utilizaremos:

In [ ]:
seed = 27912

Fijamos el tamaño del conjunto de entrenamiento:

In [ ]:
train_size = 0.7

## *1.2 Funciones auxiliares*

Estas son las funciones axiliares que hemos creado para esta práctica y que hemos añadido a nuestro utils

In [ ]:
#Funcion para realizar gráficos de caja
def plot_boxplot(data):
    
    var = data.columns
    data = widgets.fixed(data)

    widgets.interact(_plot_boxplot, data=data, var=var)

#Función auxiliar para gráficos de caja    
def _plot_boxplot(data, var):
    return data[var].iplot(kind="box")

#Función para calcular el porcentaje de ceros en variables que no deberían de tener ceros
def ZeroCount(Data, param):
    for s in param:
        aux=Data[s]
        zeros=aux.astype(bool).sum(axis=0)
        totalval=np.product(aux.shape)
        result= (1-(zeros/totalval)) * 100    
        #print(result)
        print(f"El porcentaje de ceros en la variable {s} es del {result:.2f}% ")
        

#Función para calcular el porcentaje de valores nulos en variables
def MissingValuesCount(Data, param):
    Nan= Data.isnull().sum()
    for s in param:
        Aux =Data[s]
        TotalVal= np.product(Aux.shape)
        NanSum = Aux.isnull().sum()
        result= (NanSum/TotalVal)*100  
        print(f"El porcentaje de valores nulos en la variable {s} es del {result:.2f}% ")


#Función para eliminar outliers
def outlier_rejection(X, y, seed):
    model = IsolationForest(random_state=seed)
    model.fit(X)
    y_pred = model.predict(X)
    return X[y_pred == 1], y[y_pred == 1]


#Función para realizar la evaluacion de bases de datos medicas
def EvaluationWClassRepo(model,
             X_train, X_test,
             y_train, y_test):
    
    clf = model.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    
    d = dict(enumerate(yTitanic.cat.categories))
    a="% s" % d.get(0)
    b="% s" % d.get(1)
    labels=[a,b]
    print(classification_report(y_test, y_pred, target_names=labels))
    
    disp = plot_confusion_matrix(clf, X_test, y_test)
    
    accuracy= accuracy*100

    disp.ax_.set_title(f" Tasa de precisión = {accuracy:.2f}"+"%")
    


# 2. Acceso y almacenamiento de datos

## *2.1 Breast cancer Winsconsin*

La base de datos Breast cancer Winscosin es el resultado del análisis de una imagen digitalizada de un aspirado con aguja fina (FNA) de una masa mamaria. El análisis se realiza teniendo en cuenta las distitnas variables que maneja la base de datos las cuales son:

1. ID number: Es un número creciente que servirá de identificador para cada uno de los casos de la base de datos.
2. Diagnosis: Variable que guardará el resultado del análisis de la masa mamaria. Si tiene el valor B en caso de que la masa sea benignea y M en caso de que la masa sea maligna. Esta variable será la que tomemos como variable predictora.
3. radius: Variable real que guardará la media de las distancias desde el centro hasta los puntos del perímetro de la masa.
4. texture: Variable real que guardará la desviación estándar de los valores de la escala de grises de la masa.
5. perimeter: Variable real que guardará el perímetro de la masa.
6. area: Variable real que guardará el perímetro de la masa.
7. smoothness: Variable real que guardará la variación local en longitudes de radio de la masa.
8. compactness: Variable real resultante de la operación perimeter^2 / area - 1.0 (los valores de la variables son los de la masa)
9. concavity: Variable real que representa la severidad de las porciones cóncavas del contorno de la masa.
10. concave points: Variable real que represente el numero de porciones cóncavas  del controno de la masa.
11. symmetry: Variable real que representa la simetria de la masa.
12. fractal dimension ("coastline approximation" - 1): Variable real que representa la aproximación de la linea costera de la masa

Las dos primeras variables son variables de información y el resto son variables que se computan para cada caso de masa mamaria analizada. Para estas últimas variables habrá tres tipos para cada uno de los casos:

    -Variable_mean: Guardará el valor medio de la variable.
    -Variable_se: Guardará el error estándar de la variable.
    -Variable_worst: Guardará el peor valor (media de los tres valores más grandes) de la variable.

Como conclusion la base de datos se utilizará en nuestro estudio para generar un modelo que prediga según unos valores si la masa mamaria analiza es B (Benigna) o M (Maligna)


Cargamos la base de datos Breast cancer wisconsin  tratando la variable id como indice y la variable diagnosis como variable objetivo. Finalmente mostramos una muestra del conjunto de datos cargados

In [ ]:
filepathWisconsin = "../input/breast-cancer-wisconsin-data/data.csv"
indexWisconsin = "id"
targetWisconsin = "diagnosis"
dataWisconsin = utils.load_data(filepathWisconsin, indexWisconsin, targetWisconsin)
dataWisconsin.sample(5, random_state=seed)

Si realizamos una observación de este muestra podemos darnos cuenta de que destaca una variable llamada Unnamed 32 ya que en la muestra todos los sus valores son nulos (NaN). Si nos descargamos el fichero .csv de la base de datos Breast Cancer Wisconsin, podemos observar que esta variable es ruido ya que se ha introducido por error a la hora de cargar el conjunto de datos pues en el fichero se ha introducido una coma de más y al cargarlo esa coma la toma como otra variable, para ser exactos como la variable Unnamed32.
Por ello lo que vamos a hacer a continuación es eliminar dicha columna (mediante el método de pandas drop) del conjunto de datos dataWisconsin y enseñaremos otra muestra de dicho conjunto de datos para confirmar si se han borrado correctamente.

In [ ]:
dataWisconsin = dataWisconsin.drop(dataWisconsin.columns[31], axis = 'columns')
dataWisconsin.sample(5, random_state=seed)

En el siguiente paso dividimos el conjunto de datos en dos subconjuntos, uno con las variables predictoras (X) y otro con las variables objetivo (Y). Después mostramos una muestra (sample) de los subconjuntos creados.

In [ ]:
(XWisconsin, yWisconsin) = utils.divide_dataset(dataWisconsin, target="diagnosis")

In [ ]:
XWisconsin.sample(5, random_state=seed)

In [ ]:
yWisconsin.sample(5, random_state=seed)

A continuación vamos a dividir nuestros subconjunto de datos en otros dos:
*  uno que sirva como muestra de entrenemiento (XWisconsin_train y yWisconsin_train) (70% del subconjunto inicial)
*  el otro que sirva como muestra de prueba (XWisconsin_test y yWisconsin_test) (30% del subconjunto incial)

Esta división la realizaremos con el método train_test_split.

Finalmente mostraremos una muestra de cada uno de los subconjuntos obtenidos de forma aleatoria (utilizando la semilla que hemos definido al principio de la práctica).

In [ ]:
(XWisconsin_train, XWisconsin_test, yWisconsin_train, yWisconsin_test) = train_test_split(XWisconsin, yWisconsin,
                                                      stratify=yWisconsin,
                                                      random_state=seed,
                                                      train_size=train_size)

In [ ]:
XWisconsin_train.sample(5, random_state=seed)

In [ ]:
XWisconsin_test.sample(5, random_state=seed)

In [ ]:
yWisconsin_train.sample(5, random_state=seed)

In [ ]:
yWisconsin_test.sample(5, random_state=seed)

## *2.2 Pima Indians diabetes*

La base de datos Pima Indians diabetes tiene el objetivo de predecir de forma diagnóstica si un paciente tiene o no diabetes, basándose en ciertas mediciones ya realizadas e incluidas en la base de datos. Algunas restricciones se han establecido en la seleccion de las instacias para la base de datos. En particular, todos los pacientes son mujeres con al menos 21 años con herencia del pueblo Pima (que és un grupo de indígenas de Estados Unidos que viven en Arizona). Las distitnas variables que maneja la base de datos son las siguientes:

1. Pregnancies: Número entero que indica el número de veces que ha estado embarazada la paciente.
2. Glucose: Número entero que indica la concentración de glucosa en plasma de la paciente tras 2 horas de que se la haya realizado una prueba oral de tolerancia a la glucosa.
3. BloodPresure: Número entero que indica la presión arterial diastólica de la paciente en mm/hg.
4. SkinThickness: Número entero que indica el espesor del pliegue cutáneo del triceps de la paciente en mm
5. Insulin: Número real que indica el suelo insulino tras 2 horas de la paciente en mu U/ml
6. BMI: Número entero que indica el índice de masa corporal de la paciente dado por la divisón del peso en kg entre la altura en metros al cuadrado
7. DiabetesPedigreeFunction: Número real que indica el resultado de la función de pedigree de la función.
8. Age: Número entero que indica la edad del paciente.
9. Outcome: Número categórico numérico que puede tener los valores 1 o 0. Esta variable tomará el valor 1 en el caso de que la paciente tenga diabetes y tomará el valor 0 en el caso de que la paciente no tenga diabetes.

La variable objetivo de este problema sería outcome, mientras que el resto serán variables predictoras.
Como conclusión, recordar que nuestra base de datos tendrá la finalidad de que en base a los valores de las variables predictoras, intentará diagnosticar si un paciente tiene diabetes o no "rellenando" el valor de la variable objetivo Outcome.



Lo primero que vamos a hacer es cargar la base de datos utilizando Outcome como variable objetivo y como la base de datos carece de una variable que se pueda utilizar como índice, no utilizaremos ninguna variable (None) como variable indice ya que en ese caso, el método load_data generá un indice de forma automática.

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"
indexDiabetes = None
targetDiabetes = "Outcome"
dataDiabetes = utils.load_data(filepath, indexDiabetes, targetDiabetes)
dataDiabetes.sample(5, random_state=seed)

En el siguiente paso dividimos el conjunto de datos en dos subconjuntos, uno con las variables predictoras (X) y otro con las variables objetivo (Y). Después mostramos una muestra (sample) de los subconjuntos creados.

In [ ]:
(XDiabetes, yDiabetes) = utils.divide_dataset(dataDiabetes, target="Outcome")

In [ ]:
XDiabetes.sample(5, random_state=seed)

In [ ]:
yDiabetes.sample(5, random_state=seed)

A continuación vamos a dividir nuestros subconjunto de datos en otros dos:
*  uno que sirva como muestra de entrenemiento (XDiabetes_train y yDiabetes_train) (70% del subconjunto inicial)
*  el otro que sirva como muestra de prueba (XDiabetes_test y yDiabetes_test) (30% del subconjunto incial)

Esta división la realizaremos con el método train_test_split.

Finalmente mostraremos una muestra de cada uno de los subconjuntos obtenidos de forma aleatoria (utilizando la semilla que hemos definido al principio de la práctica).

In [ ]:
(XDiabetes_train, XDiabetes_test, yDiabetes_train, yDiabetes_test) = train_test_split(XDiabetes, yDiabetes,
                                                      stratify=yDiabetes,
                                                      random_state=seed,
                                                      train_size=train_size)

In [ ]:
XDiabetes_train.sample(5, random_state=seed)

In [ ]:
XDiabetes_test.sample(5, random_state=seed)

In [ ]:
yDiabetes_train.sample(5, random_state=seed)

In [ ]:
yDiabetes_test.sample(5, random_state=seed)

## *2.3 Titanic*

La base de datos Titanic es una base de datos que recoge ciertas características en sus variables sobre los distintos pasajeros del único y famoso viaje del Titanic. Dichas variables son las siguientes:

1. PassengerId: Variable numérica entera creciente que actuará de indice de los distintos casos de la base de datos.
2. Survived: Variable categórica numérica que servirá de variable objetico y que podrá tener los valores 0 o 1, 0 en el caso de que el pasajero no sobrevivierá en el viaje y 1 en el caso de que el pasajero sobreviviera. 
3. Pclass: Variable categórica numérica que servirá para indicar la clase del pasajero. La variable podrá tener los valores 1,2 o 3; 1 en el caso de que el pasajero se alojase en primera clase, 2 en el caso de que el pasajero se alojase en segunda clase y 3 en el caso de que el pasajero se alojase en tercera clase.
4. Name: Variable string de valores únicos que tendrá el valor del nombre de los distintos pasajeros de los casos de la base de datos.
5. Sex: Variable categórica que servirá para indicar el género del pasajero. La variable podrña tener los valores male o female, male en el caso de que el género del pasajero sea masculilno y female en el caso de que el género del pasajero sea femenino.
6. Age: Variale entera que servirá para indicar la edad del pasajero.
7. SibSp: Variable entera que servirá para indicar el número de hermanos (hermano, hermana, hermanastro o hermanastra) o de parejas (marido o mujer, las prometidas y amantes se ignoraron) que tiene el pasajero en el viaje.
8. pArch: Variable entera que servirá para indicar el número de parientes que tiene el pasajero en el viaje (madre, padre, hija, hijo, hijastra, hijastro)
9. Ticket: Variable string de valores único que servirá para indicar el número de billete que tiene el pasajero. 
10. Fare: Variable real que servirá para indicar la tarifa por la que ha pagado el pasajero por su billete.
11. Cabin: Variable string que servirá para indicar la cabina en la que viajaba el pasajero durante el viaje.
12. Embarked: Variable categórica que podrá tomar los valores C, Q o S en caso de que el pasajero embarcase en los puertos de Cherbourg, Queenstown o Southampton respectivamente.


La variable que usaremos como variable objetivo será Survived, la variable que usaremos como indice será PassengerId y el resto serán variables predictoras.
Como conclusión, decir que esta base de datos se utilizará en nuestro estudio para generar modelos que prediga si un pasajero sobrevivirá al viaje o no dependiendo de ciertas características (que serán los valores de las variables predictoras).

Cargamos la base de datos, pero al tener esta implicita tres archivos .csv utilizaremos únicamente el fichero train.csv pues es el único que posee todas las variables predictoras que reccogen las características de los pasajeros. Utilizaremos la variable PassenferId como índice y Survived como variable predictora y para cargar la base de datos utilizaremos el método del utils load_data.

In [ ]:
indexTitanic="PassengerId"
targetTitanic="Survived" 
filepathTitanic="../input/titanic/train.csv"
dataTitanic = utils.load_data(filepathTitanic, indexTitanic, targetTitanic)
dataTitanic.sample(5, random_state=seed)


En el siguiente paso dividimos el conjunto de datos en dos subconjuntos, uno con las variables predictoras (X) y otro con las variables objetivo (Y). Después mostramos una muestra (sample) de los subconjuntos creados.

In [ ]:
(XTitanic, yTitanic) = utils.divide_dataset(dataTitanic, target="Survived")

In [ ]:
XTitanic.sample(5, random_state=seed)

In [ ]:
yTitanic.sample(5, random_state=seed)

A continuación vamos a dividir nuestros subconjunto de datos en otros dos:
*  uno que sirva como muestra de entrenemiento (XTitanic_train y yTitanic_train) (70% del subconjunto inicial)
*  el otro que sirva como muestra de prueba (XTitanic_test y yTitanic_test) (30% del subconjunto incial)

Esta división la realizaremos con el método train_test_split.

Finalmente mostraremos una muestra de cada uno de los subconjuntos obtenidos de forma aleatoria (utilizando la semilla que hemos definido al principio de la práctica).

In [ ]:
(XTitanic_train, XTitanic_test, yTitanic_train, yTitanic_test) = train_test_split(XTitanic, yTitanic,
                                                      stratify=yTitanic,
                                                      random_state=seed,
                                                      train_size=train_size)

In [ ]:
XTitanic_train.sample(5, random_state=seed)

En estas dos muestra que hemos obtenido ya podemos observar valores nulos (NaN) en algunas variables que es un dato que vamos a tener que tener en cuenta a la hora de realizar el análisis exploratorio y el psoterior preprocesamiento de datos.

In [ ]:
XTitanic_test.sample(5, random_state=seed)

In [ ]:
yTitanic_train.sample(5, random_state=seed)

In [ ]:
yTitanic_test.sample(5, random_state=seed)

### **Preeliminares del analisis exploratiro**



Antes de empezar el analisis exploratorio obtendremos los conjuntos de entrenamiento y de test para todas las bases de datos sobre las que realizaremos el análisis exploratorio. Para ello volvemos a unir (usando el método join_dataset del fichero utils) los conjuntos de variables predictoras con la variable clase de los subconjuntos de entrenamiento y de test de todas las bases de datos.

In [ ]:
dataWisconsin_test= utils.join_dataset(XWisconsin_test,yWisconsin_test)
dataWisconsin_train= utils.join_dataset(XWisconsin_train,yWisconsin_train)

dataDiabetes_test= utils.join_dataset(XDiabetes_test,yDiabetes_test)
dataDiabetes_train= utils.join_dataset(XDiabetes_train,yDiabetes_train)

dataTitanic_test= utils.join_dataset(XTitanic_test,yTitanic_test)
dataTitanic_train= utils.join_dataset(XTitanic_train,yTitanic_train)

# 3. Analisis exploratorio

## *3.1 Breast cancer Winsconsin*

### **Descripción del conjunto**

Tendremos que tener conocimento de:
* Numeros de casos
* Tipos de variables

Primero mostramos el número de casos y variables del conjunto de datos de entrenamiento con el metodo .shape

In [ ]:
dataWisconsin_train.shape

Podemos observar que el conjunto de datos tiene 398 casos y 31 variables,siendo estas quizas demasiadas para que se reprensenten con claridad en algunos de los gráficos que vamos a utilizar

In [ ]:
dataWisconsin_train.info(memory_usage=False)

Donde podemos observar que cada de las 31 variables que hay 30 son del tipo real y una que es del tipo categorico (Que es la variable que usaremos como variable objetivo, diagnosis)

A continuación mostramos los distitnos valores que pueden tomar nuestras variables categóricas con el método .categories:

In [ ]:
yWisconsin_train.cat.categories

Podemos observar que diagnosis es una variable categórica que puede tomar el valor B en caso de que el resultado del diagnosis de la masa del paciente sea benigna y M en casao de que el resultado del diagnosis de la masa del paciente sea maligno.

### **Visualización de las variables**

Vamos a empezar realizando un histograma sobre el conjunto de datos de entrenamiento:

In [ ]:
utils.plot_histogram(dataWisconsin_train)

Tras observar el histograma que hemos obtenido sobre el conjunto de datos de entrenamiento, podemos sacar las siguientes conclusiones:
* La primera conclusión que podemos sacar esque cada una de las variables perteneciente a un grupo (mean, se o worst) poseen una distribución similar, con alguna excepción. Por ejemplo, las variables del grupo del valor medio (mean), tienen una distribución normal con tendencia central en forma de campana de gauss, a excepción, de las variables area_mean, compactness_mean, concavity_mean y concave points_mean, que tienen una distribución con tendencia exponencial decreciente. Por otra parte, las variables del grupo del error medio (se) poseen una distribución exponencial decreciente apreciable en todas las variables del grupo. Finalmente, las variables del grupo del peor valor (worst) tienen una distribución normal con tenencia central en forma de campana de gauss, a excepción, de las variables area_worstm compactness_worst, concavity_worts y fractal dimension_worst.
* La segunda conclusión que podemos sacar esque entre las 31 variables, no hemos observado que haya presencia de datos no válidos ni indicios de outliers. No obstante; durante la realización de este análisis exploratorio realizaremos un diagrama de caja para cada una de las variables para analizar de forma más detallada la presencia de outliers.

A continuacion vamos a realizar un diagrama de barras:

In [ ]:
utils.plot_barplot(dataWisconsin_train)
B,M= yWisconsin_train.value_counts()
print('Numero de masas benigneas: ',B)
print('Numero de masas malignas : ',M)

Observamos que en nuestro conjunto de datos entrenamientos hay más casos en el que el diagnóstico final fue benigneo(250)que en el que el diagnóstico final fue maligno (148), esto quiere decir que el problema está desbalanceado.

Continuaremos con el análisis exploratorio realizando un diagrama de puntos:

In [ ]:
utils.plot_pairplot(dataWisconsin_train, target="diagnosis")

Lo único que podemos observar es lo que hemos mencionado al principio de este análisis, al ser 30 variables predictoras, algunas representaciones gráficas pueden no verse con claridad y este es un ejemplo. Por ello lo que vamos a hacer es dividir el diagrama de puntos entre tres, uno para cada uno de los tres tipos que pueden tener las variables predictoras: mean, se y worst.

In [ ]:
mean_train = dataWisconsin_train.iloc[:,[0,1,2,3,4,5,6,7,8,9,30]]
se_train = dataWisconsin_train.iloc[:,[10,11,12,13,14,15,16,17,18,19,30]]
worst_train = dataWisconsin_train.iloc[:,[20,21,22,23,24,25,26,27,28,29,30]]
utils.plot_pairplot(mean_train, target="diagnosis")

In [ ]:
utils.plot_pairplot(se_train, target="diagnosis")

In [ ]:
utils.plot_pairplot(worst_train, target="diagnosis")

Respecto a estas graficas podemas darnos cuenta que las variables radius_mean, perimeter_mean, area_mean estan muy relacionados, obviamente eso puede ser por que para obtener el perimetro y el area es necesario saber el radio. Puede que sea factible descartar estas variables y quedarnos con solo radio ya que al estar muy relacionados dicha eliminación no afectaría negativamente a la generación de modelos y consideramos que es mejor eliminar 3 variables que una para tener menos variables que gestionar durante el proceso de modelado.
También podemos observar que la mejor forma de realizar una discretización óptima sería utilizando dos contenedores y quizas la mejor estrategia sería utilizar KMeans, pero hay tantos casos que no se puede estar seguro a simple vista.

Para comprobar algunas de las conclusiones que hemos realizado en el apartado anterior vamos a generar una matriz de correlación:

Al igual que para el gráfico anterior vamos a realizar tres matrices distintas una para cada uno de los tipos que pueden tener las variables predictoras; mean,se y worst.

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(mean_train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(se_train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(worst_train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

Las conclusiones que podemos sacar es que existe cierta relación entre las variables del conjunto de variables radius_mean, perimeter_mean y area_mean y entre las variables del conjunto de variables concavity_mean, concave points_mean y concavity_mean. Además de que existe cierta relación entre estos dos conjuntos de variables, por lo tanto durante el preprocesamiento deberiamos de tener en cuenta estas conclusiones y decidir si tenemos en cuenta estas variables para modelar el problema.

Diagrama de caja

In [ ]:
utils.plot_boxplot(dataWisconsin_train)

Las conclusiones que podemos sacar de estos diagramas de caja esque se indentifican la presencia de outliers en distintas variables (se puede ver claramente por ejemplo en los diagramas de caja de las variables radius_se y area_worst),por lo que se probará a utilizar un estimador que elimine los outliers para comprobar si mejora los resultados de los distintos modelos.

## *3.2 Pima Indians diabetes*

### **Descripción del conjunto**

Tendremos que tener conocimento de:
* Numeros de casos
* Tipos de variables

Primero mostramos el número de casos y variables del conjunto de datos de entrenamiento con el metodo .shape

In [ ]:
dataDiabetes_train.shape

El número de casos del conjunto de datos es 537 mientras que el número de variables es 9

A continuación mostramos el tipo de cada una de las variables del conjunto de datos con el método .info:

In [ ]:
dataDiabetes_train.info(memory_usage=False)

Donde podemos observar que de las 9 variables, todas son de tipo entero excepto tres; BMI y DiabetesPedigreeFunction que son de tipo real (dato que tendremos que tener en cuenta si las modificamos en el preprocesamiento), y Outcome que es de tipo categórico y es la variable que tomaremos como variable objetivo.

A continuación mostramos los distitnos valores que pueden tomar nuestras variables categóricas con el método .categories:

In [ ]:
yDiabetes.cat.categories

Podemos observar que la variable Outcome es una variable categórica numérica que puede tomar los valores 0 o 1 en caso negativo o afirmativo de que el paciente tenga diabetes respectivamente.

### **Visualización de las variables**

Vamos a empezar realizando un histograma sobre el conjunto de datos de entrenamiento:

In [ ]:
utils.plot_histogram(dataDiabetes_train)

Si observamos la gráficas podemos observar dos detalles importantes:
* Lo primero es que no todas las variables tienen una distribución normal con tendencia central en forma de campana de gauss, solamente las variables SkinThickness, BMI, BloodPresure y Glucose; mientras que el resto de variables( Age, Pregnancies, Insulin y DiabetesPedigreeFunction) poseen una distribución con tendencia exponencial decreciente. De hecho en ciertas variables como DiabetesPedigreeFunction o Insulin, en sus gráficas podemos empezar a apreciar la aparición de outliers que comprobaremos más tarde con los gráficos de caja.
* Lo segundo es que en las gráficas se puede observar como ciertas variables toman valores perdidos, es decir toman el valor 0 cuando según la lógica de los valores que pueden tomar dichas variables sería imposible que dichas variables tuvisen como valor un 0. Dichas variables son Glucose (un paciente no puede tener 0 de glucosa), BloodPresure (un paciente no puede tener 0 de presión sanguínea), SkinThickness (la piel de un paciente debe de tener grosor), Insulin (un paciente ha de tener insulina) y BMI (el índice de masa corporal de una persona no puede ser 0).

A continuación vamos a obtener el porcentaje de ceros que tienen estas variables con el método del fichero de utilidad ZeroCount y así comproboremos la cantida de ruido que tienen estas variables y si vale la pena tener estas variables en cuenta para el preprocesamiento:

In [ ]:
param=["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

utils.ZeroCount(dataDiabetes_train,param)

La conclusión que podemos sacar esque el porcentaje de ruido que tienen variables como BloodPresure, Glucose o BMI es aceptable, mientras que el de las variables Insulin y SkinThickness (48.60% y 29.24% de ruido respectivamente) es tan alto que lo más recomendable esque no las tengamos en cuenta para generar nuestro modelos.

A continuacion vamos a realizar un diagrama de barras:

In [ ]:
utils.plot_barplot(dataDiabetes_train)
S,N= yDiabetes_train.value_counts()
print('Numero de Diabetes: ',S)
print('Numero de no Diabbetes: ',N)

Podemos observar que en nuestro conjunto de datos hay más casos en los que la variable objetivo Outcome indicaba que el paciente tenia diabetes (350) que en los que la variable Outcome indicaba que el paciente no tenia diabaetes (187). Al haber tanta diferencia entre el número de los casos en los que el paciente tiene o no diabetes podemos decir que el problema está desbalanceado.

Continuaremos con el análisis exploratorio realizando un diagrama de puntos:

In [ ]:
utils.plot_pairplot(dataDiabetes_train, target="Outcome")

Podemos observar que hay ciertas variables muy relacionadas entre si, como Age y Pregnancies, Glucose y Age o Glucose e Insulin; por lo que quizas en procesos posteriores deberiamos eliminar alguno de los miembros de estos pares de variables. Tambíen podemos darnos cuenta de que si realizamos una discretización quizas deberíamos de realizarla con 3 contenedores, el problema, esque los datos están tan juntos que en principio no podemos definir ninguna estrategia de discretización correcta para este conjunto de datos.

Para comprobar algunas de las conclusiones que hemos realizado en el apartado anterior vamos a generar una matriz de correlación:

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(dataDiabetes_train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

Al observar la matriz nos podemos dar cuenta de que el par de variables Glucose-Age y Glucose-Insulin están relacionados pero no lo suficiente para eliminar una de estas variables en el preproceasmiento; no obsatne, el par de variables Age-Pregnancies es el más relacionado del conjunto de datos y quizas deberiamos de eliminar una de estas variables durante el preprocesamiento.

Para acabar el análisis exploratorio vamos a realizar un diagrama de caja para comprobar si podrían haber outliers en las variables del conjunto de datos:

In [ ]:
utils.plot_boxplot(dataDiabetes_train)

Las conclusiones que podemos sacar de los distintos diagramas de cajas esque en las variables se aprecian cierta cantidad de valores que están demasiado distanciados de la mediana, notándose sobre todo en los diagramas de las variables Insulin y SkinThickness, por lo que se justifica el uso de un estimador para eliminar outliers para modelar este conjunto de datos.

## *3.3 Titanic*

### **Descripción del conjunto**

Tendremos que tener conocimento de:
* Numeros de casos
* Tipos de variables

Primero mostramos el número de casos y variables del conjunto de datos de entrenamiento con el metodo .shape

In [ ]:
dataTitanic_train.shape

Podemos observar que el número de casos es 623 y el número de variables es 11.

A continuación mostramos el tipo de cada una de las variables del conjunto de datos con el método .info:

In [ ]:
dataTitanic_train.info(memory_usage=False)

Si observamos el tipo de cada una de las variables podemos darnos cuenta de que 3 son de tipo entero, que 2 son de tipo real (dato que quizas deberiamos de tener en cuenta más adelante en el preprocesamiento), 1 es de tipo categórico (que será la variable que tomaremos como variable objetivo) y  5 son de tipo objeto, lo cual significará un inconveniente para la representación de datos pues no aparecerán en nuestras gráficas y también deberemos de replantearnos si estas variables objeto influirán numéricamente en la creación de nuestros modelos finales.

A continuación mostramos los distitnos valores que pueden tomar nuestras variables categóricas con el método .categories:

In [ ]:
yTitanic.cat.categories

Podemos observar que survived se trata de una variable numérica categórica que podrá tomar los valores 0 o 1 en caso negativo o afirmativo de que el pasajero sobreviviese al viaje respectivamente.

### **Visualización de las variables**

Vamos a empezar realizando un histograma sobre el conjunto de datos de entrenamiento:

In [ ]:
utils.plot_histogram(dataTitanic_train)

Las conclusiones que podemos obtener del siguiente histograma son las siguientes:
* A pesar de tener 11 variables, solo 5 aparecen en nuestro histograma, por lo tanto debemos de analizar porque las otras 6 no aparecen. Name, ticket y Cabin no aparecen debido a que son variables de tipo string y sus valores no se pueden representar en este tipo de gráficos, además de que consideramos que sus valores no aportan mucho a este tipo de problemas al ser variables de tipo string (y en el caso de Name y ticket también valores únicos) y por lo tanto no los tendremos en cuenta para desarrollar los estimadores para generar los modelos de esta base de datos. Sex y embarked no aparecen al ser variables categóricas no numéricas, por lo tanto para poder ser tenidas en cuenta a la hora de generar nuestros modelos primero tenemos que modificarlas para convertirlas en variables categóricas numéricas en el preprocesamiento.Finalmente Surivived no aparece al ser la variable objetivo.
* De las variables que representadas solo Age posee un distribución normal con tendecia central en forma de campana de gauss, mientras que Pclass posee una distribución con tendencia exponencial creciente y el resto de variables (Sibsp, Parch y Fare) poseen una distribución con tendencia exponencial decreciente.
* Al ver simplemente el histograma podemos darnos cuenta de indicios de outliers en la variable Fare,sin embargo, habrá que esperar a observar los diagramas de cajas para observar si el resto de variables poseen outliers.
* No hemos observado que haya variables con valores no válidos durante la observación del histograma.

Al realizar la carga de datos y enseñar una muestra de los datos cargados, pudimos observar qeu habias ciertas variables con valores nulos (NaN) y como no hemos podido observar en el histograma que variables poseen dichos valores nulos, vamos a recurrir al método isnull().sum para que se nos muestren todas las variables con valores nulos.

In [ ]:
dataTitanic_train.isnull().sum()

Se nos muestra que las variables con valores nulos son Age, Embarked y Cabin. Los valores nulos de las variables Age y Embarked los arreglaremos durante el preprocesamiento ya que al ser variables categóricas no numéricas las tendremos que convertir a variables categóricas numéricas y los valores nulos asumiremos que es un error de quien haya redactado la base de datos y quería poner el valor más frecuente. Los valores de la variable Cabin son distintos, ya que al tratarse de una variable de tipo string no la ibamos a tener en cuenta ya a la hora de generar los modelos. Sin embargo vamos a recurrir al método MissignValuesCount para comprobar si el porcentaje de valores nulos (ruido) es tan grande que no deberíamos de tener en cuenta estas variables durante el preprocesamiento.

In [ ]:
param=["Age","Embarked","Cabin"]
utils.MissingValuesCount(dataTitanic_train,param)


El porcentaje de valores nulos las dos variables que ya pensabamos arreglar durante el preprocesamiento (Age y Embarked) es tan inferior (menos del 20%) que las seguiremos teniendo en cuenta durante el preprocesamiento, mientras que el porcentaje de Cabin (que era una variable que no ibamos a tener en cuenta por el tipo de variable que era) es tan alto que la hubiesemos tenido que eliminar del proceso de generación de modelos aunque el tipo de la variable no hubiese sido de tipo string.

A continuacion vamos a realizar un diagrama de barras:

In [ ]:
utils.plot_barplot(dataTitanic_train)
D,S= yTitanic_train.value_counts()
print('Numero de supervivientes: ',S)
print('Numero de muertos: ',D)

Según este diagrama podemos observar que en el conjunto de datos de entrenamiento hay más casos en los que el viajero no sobrevivió (el valor de la variable Survived sería 0) que casos en los que el viajero sobrevivió (el valor de la variable Survived sería 1) por lo que podemos indicar que el problema está desbalanceado.

Continuaremos con el análisis exploratorio realizando un diagrama de puntos:

In [ ]:
utils.plot_pairplot(dataTitanic_train, target="Survived")

Podemos observar que este gráfico representa las variables que antes no se nos han mostrado, y en principio no vemos ningún par de variables muy relacionados entre sí. Lo que si que vemos con más claridad esque si realizamos una discretización lo mejor será utilizar una estrategia kmeans y con 2 contenedores ya que los casos con el mismo valor en la variable objetivo survived se encuentran más agrupados y más separados de los casos con distinto valor en la variable objetivo.

Para comprobar algunas de las conclusiones que hemos realizado en el apartado anterior vamos a generar una matriz de correlación:

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(dataTitanic_train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

Las conclusiones que podemos sacar de esta matriz esque no sólo no hay ningún par de variables muy relacionados entre sí, sino que aunque hay variables que están muy poco relacionadas entre si (con valores negativos en la matriz de correlación) tampoco hay ningún par de variables que supere el -0.5 como coecifiente de correlación. Por lo tanto podemos decir que ninguna de las variables del conjunto de datos es prescindible para realizar el posterior modelado del problema

Para acabar el análisis exploratorio vamos a realizar un diagrama de caja para comprobar si podrían haber outliers en las variables del conjunto de datos:

In [ ]:
utils.plot_boxplot(dataTitanic_train)

Las conclusiones que podemos sacar de estos gráficos esque ciertas variables tienen demasiados valores alejados de la mediana, estas variables son Sex, SibSp, Parch y Fare, siendo los diagramas de Fare y SibSp donde más se notas. Por ello consideramos que está justificado el uso de un estimador que elimine los valores outliers durante el modelado del problema.

In [ ]:
def _filter_categorical_data(data):
    """Filter the categorical data."""
    return data.select_dtypes(include="category")

def _plot_barplot(data, var):
    """Plot univariate distribution of the categorical variable."""
    # Count the relative frequency of unique values
    count = data[var].value_counts(normalize=True)

    return count.iplot(kind="bar")

def Pruebaplot_barplot(data):
    """Plot univariate distribution of the categorical data."""
    #categorical_data = _filter_categorical_data(data)

    # Add a dropdown widget to select
    # the categorical feature to plot
    var = data.columns
    data = widgets.fixed(data)

    widgets.interact(_plot_barplot, data=data, var=var)


In [ ]:
#Prueba
#Sex Farea
Notcat = dataTitanic_train.iloc[:,[3,7,10]]
#Pclass Sex Sibsp Parch
Cat = dataTitanic_train.iloc[:,[0,2,4,5,7,10]]
Pruebaplot_barplot(Cat)

# 4. Preprocesamiento de datos

En esta parte aplicaremos el preprocesamiento en base a las modificaciones que hemos visto que necesita el conjunto de datos durante el análisis exploratorio.

Empezamos el preprocesamiento definiendo el estimador que hara uso de la función outlier_rejection del fichero de utilidad utils para eliminar outliers. Este estimador se utilizará posteriormente en la generación de modelos aplicandosé al conjunto de datos.

In [ ]:
#Estimador para eliminar outliers
OutlierRejection = FunctionSampler(func=utils.outlier_rejection, kw_args={"seed":seed})

## *4.1 Breast cancer Winsconsin*

Como hemos concluido en el análisis exploratorio, de los dos conjuntos de variables perimeter_mean-area_mean-radio mean y concavepoints_mean-compactness_mean-concavity_mean están muy relacionadas las variables dentro del conjunto y un conjunto de variables con el otro, por lo tanto para el preprocesamiento, vamos a generar un estimador que elimine todos los tipos (mean,se y worst) de dos variables de cada uno de los conjuntos de variables del conjunto de datos, para ello hemos elegido arbitrariamente que las variables que eliminaremos del primer conjunto de datos serán area y perimeter y del segundo serán concave y compactness.
Para la generación del estimador vamos a utilizar el método make_column_transformer de la libreria sklearn.compose y vamos a indicar como parámetro tranformador 'drop' para que elimine la lista de variables de las que hemos hablado con anterior y hemos indicado que para el resto de variables simplemente no las toque igualando el parámetro remainder como 'passthrough'.

In [ ]:
preproceserWinsconsin = make_column_transformer(('drop',['perimeter_mean','area_mean','perimeter_se','area_se','perimeter_worst','area_worst','concave points_mean','compactness_mean',
          'concave points_se','compactness_se','concave points_worst','compactness_worst']),
                                                remainder='passthrough')


Como dato aclaratorio, también podriamos haber generado el estimador utilizando el transformador 'passthrough' y seleccionado todas las variables con las que nos queremos quedar sin igualar el parámetro remainder a passthrough, pues esto hará que las variables que no le especifiquemos las borrará del conjunto de datos; básicamente es hacer el proceso que hemos hecho con el estimador al revés.

## *4.2 Pima Indians diabetes*

Como hemos indicado en el preprocesamiento de esta base de datos lo que tendremos que hacer será sustituir los valores cero de las variables Glucose, BloodPressure y BMI, eliminar una variable del par de variables Age y Pregnancies por su alto coeficiente de correlación, eliminación de las variables Insulin y SkinThickness por la cantidad de valores perdidos que tienen y dejar el resto de variables igual.

Para la sustitución de valores cero de las variables Glucose, BloodPressure y BMI utilizaremos el estimador SimpleImputer pero teniendo en cuenta que BMI se trata de una variable real y que las otras dos se tratan de variables enteras, tendremos que generar un estimador para cada tipo de variable modificando la estrategia de búsqueda, pues aunque lo normal es utilizar la media para la sustitución de valores ruidosos, al ser Glucose y BloodPresure enteros, no se puede introducir de repente un valor con decimales (la media podría tener o no decimales) por ello el simpleImputer que afectará a las variables enteras utilizará la estrategia "most_frequent" sustituyendo los ceros por el valor más frequente que claramente será de tipo entero y sin decimales. Ambos SimpleImputer (de enteros y decimales) los "combinaremos" con un método make_column_transformer donde le pasaremos a cada estimador las variables correspondientes asignándolas al parámetro pattern del make_column_selector. Para dejar al resto de variables igual simplemente declararemos las variables que no se deben de tocar en un parámetro aparte y se introducirán en el make_column_transformer pero en vez de pasarle un estimador como parámetro introduciremos el valor 'passthrough' que simplemente dejará igual a las variables. Finalmente si queremos eliminar las variables que hemos acordado en el análisis exploratorio bastará con no incluirlas en el make_colum_transformer.

In [ ]:
#Variables para sustituir ceros
features1 = 'Glucose|BloodPressure'
features2 = 'BMI'
#Variables que no hay que tocar
features3 = 'DiabetesPedigreeFunction|Age'

#Estimador para integuers
replace0Integer_Estimator = make_pipeline(SimpleImputer(strategy="most_frequent",missing_values=0 ))
#Estimador para reales
replace0Float_Estimator = make_pipeline(SimpleImputer(strategy="mean",missing_values=0 ))

preproceserDiabetes = make_column_transformer(
    (replace0Integer_Estimator, make_column_selector(pattern= features1)),
    (replace0Float_Estimator, make_column_selector(pattern= features2)),
     ('passthrough', make_column_selector(pattern= features3)))



## *4.3 Titanic*

El preprocesamineto del conjunto de datos de la base de datos Titanic es algo complicado por ello vamos a dividir la explicación en tres distintas partes:
* Primero, tenemos que tratar las variables que hemos detectado que son categóricas puras y que también tienen valores nulos durante el análisis exploratorio, dichas variables son Sex y Embarked. Para ello crearemos un pipeline con dos estimadores, uno primero que será SimpleImputer que sustituirá los valores perdidos (NaN) usando la estrategia 'most_frequent' (que es la recomendable para variables categóricas puras) y otro OneHotEncoder que conviertirá las variables categóricas puras en categóricas numéricas.
* Después tenemos que tratar las variables categóricas numéricas y las que contienen valores perdidos (NaN), que serán las variables Age, SibSp, Parch, Fare y Pclass. Para ello crearemos un pipeline con dos estimadores, uno primero que será SimpleImputer que sustituirá los valores perdidos (NaN) usando la estrategia 'median' que sustituirá los valores perdidos por la mediana de los valores de la variable (esto es así porque dentro de las variables tenemos una mezcla de variables enteras y reales y debido a esta mezcla no podemos recurrir ni a la media ni a la moda, 'most frequent', entonces recurriremos a la mediana) y el otro estimador será un StandarScaler que intentará estandarizar las características.
* Finalmente estos dos estimadores los juntaremos en un método_make_column_tranformer donde introduciremos las dos pipelines generadas con sus corresponientes conjuntos de variables, y para eliminar las variables que hemos acordado eliminar del conjunto de datos durante el análisis exploratorio (Name, Ticket y Cabin) simplemente no las introduciremos en ninguno de los dos conjuntos de variables.

In [ ]:
#Primer pipeline categoricos: Imputas y encoder
#Segundo pipeline para numerico categoricos si hay valores perdidos media
#Las variables que no aportan al modelo directamente no se meten

#Variables categoricas y con valores perdidos
featuresCat='Sex| Embarked'
#Variables numerico categoricos y variables con valores peridos
featuresR0 = 'Age|SibSp|Parch|Fare|Pclass'



#Pipeline para las variables categóricas puras que se encargara de imputar los valores nulos y realizar un encoder
Cat_Estimator = make_pipeline(SimpleImputer(strategy='most_frequent'),
                                    OneHotEncoder(handle_unknown='ignore'))

#Pipeline para numéricos y numéricos categóricos que se encarga de eliminar los valores nulos
Num_Estimator = make_pipeline(SimpleImputer(strategy='median'), StandardScaler())


preproceserTitanic = make_column_transformer(
    (Cat_Estimator, make_column_selector(pattern=featuresCat)),
    (Num_Estimator, make_column_selector(pattern=featuresR0))
    
)



# 5. Algoritmos de clasificación

En esta parte generaremos los modelos que vamos a evaluar y las pipelines que saldrán como resultado de aplicar a dichos modelos los estimadores que hemos creado en el apartado anterior (que también evaluaremos posteriormente).

Los modelos que evaluaremos serán el modelo zero_r y el modelo de arbol de decision que usará la semilla que estamos durante todo el problema.

In [ ]:
#Generación del modelo zero_r
zero_r_model = DummyClassifier(strategy="most_frequent")
#Generación del modelo de árbol de decision
tree_model = DecisionTreeClassifier(random_state=seed)

## *5.1 Breast cancer Winsconsin*

Para empezar generamos los distintos discretizadores que aplicaremos al modelo de árbol de decisión. Como hemos mencionado en el análisis exploratorio utilizaremos 2 contenedores y probaremos todas las estrategias al no tener claro del todo cual hay que utilizar.

In [ ]:
discretizerUWinsconsin = KBinsDiscretizer(n_bins=2, strategy="uniform")
discretizerQWinsconsin = KBinsDiscretizer(n_bins=2, strategy="quantile")
discretizerKWinsconsin = KBinsDiscretizer(n_bins=2, strategy="kmeans")


A continuación le aplicaremos al modelo de árbol de decisión el estimador con el preprocesamiento que hemos desarrolado en el apartado anterior. No se lo aplicamos al modelo zeroR porque no le afectaria. Tambien le aplicamos el estimador al árbol de decisión discretizado con 2 contenedores y todas las posibles estrategias de discretización.

In [ ]:

#Modelo de árbol de decisón con eliminación de variables predictoras muy relacionadas
preprocess_treemodel_Winsconsin=make_pipeline(preproceserWinsconsin, tree_model)

#Modelo de árbol de decisión con tres tipos de discretizaciones y con eliminación
# de variable predictoras muy relacionadas
preprocess_treemodel_discretizeU_Winsconsin=make_pipeline(preproceserWinsconsin,discretizerUWinsconsin,tree_model)
preprocess_treemodel_discretizeQ_Winsconsin=make_pipeline(preproceserWinsconsin,discretizerQWinsconsin,tree_model)
preprocess_treemodel_discretizeK_Winsconsin=make_pipeline(preproceserWinsconsin,discretizerKWinsconsin,tree_model)

Finalmente hemos generado cuatro pipelines extra, que son las mismas cuatro pipelines anteriores pero añadiéndoles un estimador que elimina outliers. Este estimador lo hemos añadido en cuatro pipelines extra aparte en vez de integrarlo todo junto porque sabemos que el proceso de eliminación de outliers puede generar sobreajuste y empeorar los resultados de la evaluación; por lo tanto vamos a evaluar tanto las pipelines sin eliminación de outliers como las que si que lo hacen y comparar los resultados obtenidos en el siguiente apartado.

In [ ]:

#Modelo de árbol de decisón con eliminación de variables predictoras muy relacionadas 
#eliminación de outliers
extra_preprocess_treemodel_Winsconsin=make_pipeline(OutlierRejection, preproceserWinsconsin, tree_model)

#Modelo de árbol de decisión con tres tipos de discretizaciones y con eliminación
# de variable predictoras muy relacionadas y eliminación de outliers
extra_preprocess_treemodel_discretizeU_Winsconsin=make_pipeline(preproceserWinsconsin,OutlierRejection,discretizerUWinsconsin,tree_model)
extra_preprocess_treemodel_discretizeQ_Winsconsin=make_pipeline(preproceserWinsconsin,OutlierRejection,discretizerQWinsconsin,tree_model)
extra_preprocess_treemodel_discretizeK_Winsconsin=make_pipeline(preproceserWinsconsin,OutlierRejection,discretizerKWinsconsin,tree_model)



## *5.2 Pima Indians diabetes*

Para empezar generamos los distintos discretizadores que aplicaremos al modelo de árbol de decisión. Como hemos mencionado en el análisis exploratorio utilizaremos 3 contenedores y probaremos todas las estrategias al no tener claro del todo cual hay que utilizar

In [ ]:
discretizerUDiabetes = KBinsDiscretizer(n_bins=3, strategy="uniform")
discretizerQDiabetes = KBinsDiscretizer(n_bins=3, strategy="quantile")
discretizerKDiabetes = KBinsDiscretizer(n_bins=3, strategy="kmeans")

A continuación le aplicaremos al modelo de árbol de decisión el estimador con el preprocesamiento que hemos desarrolado en el apartado anterior. No se lo aplicamos al modelo zeroR porque no le afectaria. Tambien le aplicamos el estimador al árbol de decisión discretizado con 3 contenedores y todas las posibles estrategias de discretización.

In [ ]:
#Modelo de árbol de decisión con sustitución de ceros
preprocess_treemodel_Diabetes=make_pipeline(preproceserDiabetes, tree_model)

#Modelo de árbol de decisión con tres tipos de discretizaciones y sustitución de ceros
preprocess_treemodel_discretizeU_Diabetes=make_pipeline(preproceserDiabetes,discretizerUDiabetes,tree_model)
preprocess_treemodel_discretizeQ_Diabetes=make_pipeline(preproceserDiabetes,discretizerQDiabetes,tree_model)
preprocess_treemodel_discretizeK_Diabetes=make_pipeline(preproceserDiabetes,discretizerKDiabetes,tree_model)


Finalmente hemos generado cuatro pipelines extra, que son las mismas cuatro pipelines anteriores pero añadiéndoles un estimador que elimina outliers. Este estimador lo hemos añadido en cuatro pipelines extra aparte en vez de integrarlo todo junto porque sabemos que el proceso de eliminación de outliers puede generar sobreajuste y empeorar los resultados de la evaluación; por lo tanto vamos a evaluar tanto las pipelines sin eliminación de outliers como las que si que lo hacen y comparar los resultados obtenidos en el siguiente apartado.

In [ ]:

#Modelo de árbol de decisión con sustitución de ceros y eliminación de outliers
extra_preprocess_treemodel_Diabetes=make_pipeline(preproceserDiabetes,OutlierRejection, tree_model)

#Modelo de árbol de decisión con tres tipos de discretizaciones, sustitución de ceros y eliminación de outliers
extra_preprocess_treemodel_discretizeU_Diabetes=make_pipeline(preproceserDiabetes,OutlierRejection,discretizerUDiabetes,tree_model)
extra_preprocess_treemodel_discretizeQ_Diabetes=make_pipeline(preproceserDiabetes,OutlierRejection,discretizerQDiabetes,tree_model)
extra_preprocess_treemodel_discretizeK_Diabetes=make_pipeline(preproceserDiabetes,OutlierRejection,discretizerKDiabetes,tree_model)

## *5.3 Titanic*

Para empezar generamos los distintos discretizadores que aplicaremos al modelo de árbol de decisión. Como hemos mencionado en el análisis exploratorio utilizaremos 2 contenedores y probaremos todas las estrategias al no tener claro del todo cual hay que utilizar

In [ ]:
discretizerUTitanic = KBinsDiscretizer(n_bins=2, strategy="uniform")
discretizerQTitanic = KBinsDiscretizer(n_bins=2, strategy="quantile")
discretizerKTitanic = KBinsDiscretizer(n_bins=2, strategy="kmeans")

A continuación le aplicaremos al modelo de árbol de decisión el estimador con el preprocesamiento que hemos desarrolado en el apartado anterior. No se lo aplicamos al modelo zeroR porque no le afectaria. Tambien le aplicamos el estimador al árbol de decisión discretizado con 3 contenedores y todas las posibles estrategias de discretización.

In [ ]:
#El preprocesamiento al que nos referiremos a continuación consistará de dos partes, una en la eliminación conversión de 
#categóricas y la sustitución de sus valores anómalos por un 0 y la otra en la sutitución de los valores anómalos del resto
#de variables y la estandarización de estas variables

#Modelo de árbol de decisión con preprocesamiento
preprocess_treemodel_Titanic = make_pipeline(preproceserTitanic, tree_model)

#Modelo de árbol de decisión con preprocesamiento y con tres tipos de discretizaciones
preprocess_treemodel_discretizeU_Titanic=make_pipeline(preproceserTitanic,discretizerUTitanic,tree_model)
preprocess_treemodel_discretizeQ_Titanic=make_pipeline(preproceserTitanic,discretizerQTitanic,tree_model)
preprocess_treemodel_discretizeK_Titanic=make_pipeline(preproceserTitanic,discretizerKTitanic,tree_model)

Finalmente hemos generado cuatro pipelines extra, que son las mismas cuatro pipelines anteriores pero añadiéndoles un estimador que elimina outliers. Este estimador lo hemos añadido en cuatro pipelines extra aparte en vez de integrarlo todo junto porque sabemos que el proceso de eliminación de outliers puede generar sobreajuste y empeorar los resultados de la evaluación; por lo tanto vamos a evaluar tanto las pipelines sin eliminación de outliers como las que si que lo hacen y comparar los resultados obtenidos en el siguiente apartado.

In [ ]:

#Modelo de árbol de decisión con preprocesamiento y eliminaciónde outliers
extra_preprocess_treemodel_Titanic=make_pipeline(preproceserTitanic,OutlierRejection, tree_model)

#Modelo de árbol de decisión con preprocesamiento, con tres tipos de discretizaciones 
#y eliminación de outliers
extra_preprocess_treemodel_discretizeU_Titanic=make_pipeline(preproceserTitanic,OutlierRejection,discretizerUDiabetes,tree_model)
extra_preprocess_treemodel_discretizeQ_Titanic=make_pipeline(preproceserTitanic,OutlierRejection,discretizerQDiabetes,tree_model)
extra_preprocess_treemodel_discretizeK_Titanic=make_pipeline(preproceserTitanic,OutlierRejection,discretizerKDiabetes,tree_model)

# 6. Evaluacion de modelos

El proceso de evaluación de modelos lo realizaremos siguiendo los siguientes paso para cada uno de los subapartados siguientes (uno para cada uno de las bases de datos que hemos tratado):
1. Mencion de la métrica que utilizaremos para elegir el mejor modelo de los evaluados en base a la base de datos que utilizaremos
2. Mostrar el resultado de la evaluación sobre el conjunto de datos corresponiente de los dos modelos sin inclusión de estimadores extra ,el resultado de la evaluación sobre el conjunto de datos correspondiente de las 4 pipelines resultado de aplicarle al árbol de decisión la discretización y el preprocesamiento y el resultado de la evaluación sobre el conjunto de datos correspondiente de 4 pipelines resultado de añadirle a las cuatro pipelines anteriores un estimador de eliminación de outliers. La evalución se realizará utilizando la función del fichero de utilidad utils EvaluationWClassRepo
3. Redacción de una pequeña conclusión que haga referencia al mejor resultado obtenido y las consecuencias supone dicho resultado.

## *6.1 Breast cancer Winsconsin*

Teniendo en cuenta que la base de datos Breast cancer winsconsin surge de un problema médico y por ello, la métrica normalmente utilizada para estos problemas y la que tenemos que emplear para la evaluación es el recall o tasa de verdaderos positivos. Dicha tasa la podemos ver en la estructura classsification report que generará el método EvalutationWClassRepo. 
Finalmente, al utilizarse la base de datos para predecir el resultado del análisis de una masa mamaria, consideraremos que dicho análisis da positivo si el diagnóstico es maligno,es decir; la tasa de veraderos positivos se observará teniendo en cuenta que un positivo quiere decir que la masa mamaria es maligna.

> ### *Modelos sin preprocesamiento*

In [ ]:
utils.EvaluationWClassRepo(zero_r_model,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(tree_model,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

> ### *Modelos con el preprocesamiento desarrollado anteriormente*

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeU_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeQ_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeK_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)


> ### *Modelos con el preprocesamiento extra*

In [ ]:
EvaluationWClassRepo(extra_preprocess_treemodel_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeU_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeQ_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeK_Winsconsin,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

> ### *Conclusiones*

Las conclusiones que hemos obtenido tras analizar los resultados de la evaluación de modelos son las siguientes:
* Los dos mejores modelos son el modelo del árbol de decisión con el preprocesamiento básico y sin discretizar y el modelo del árbol de decisión con el preprocesamiento básico y con una discretización de igual frecuencia. Ambos tienen una tasa de verdaderos positivos del 0.94.
* Si tenemos en cuenta la tasa de verdaderos positivos media (La media de la tasa de verdaderos positivos para positivo benigneo y positivo maligno) el mejor modelo sería el árbol de decisión con el preprocesamiento básico y sin discretizar con un tasa de verdaderos positivos media del 0.94.
* En general hemos obtenido una tasa de verdaderos positivos alta para todos nuestros modelos (el más bajo sin contar el modelo ZeroR ha sido 0.64).
* La eliminación de outliers solo ha sobreajustado el modelo para este conjunto de datos ya que todos los modelos que han eliminado outliers han tenido peores resultados que su contraparte sin eliminación de outliers.
* La mejor discretización para el modelo de árboles de decisión es la de igual frecuencia, eliminado o no outliers.

## *6.2 Pima Indians diabetes*

Teniendo en cuenta que la base de datos Pima Indians diabetes surge de un problema médico y por ello, la métrica normalmente utilizada para estos problemas y la que tenemos que emplear para la evaluación es el recall o tasa de verdaderos positivos. Dicha tasa la podemos ver en la estructura classsification report que generará el método EvalutationWClassRepo. Finalmente, al utilizarse la base de datos para predecir de forma diagnóstica si un paciente tiene diabete o no, consideraremos que dicho análisis da positivo si la variable objetivo Outcome tiene el valor 1, es decir; la tasa de verdaderos positivos se observará teniendo en cuenta que un positivo quiere decir que el paciente tiene diabetes.

> ### *Modelos sin preprocesamiento*

In [ ]:
utils.EvaluationWClassRepo(zero_r_model,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(tree_model,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

> ### *Modelos con el preprocesamiento desarrollado anteriormente*

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeU_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeQ_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeK_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

> ### *Modelos con el preprocesamiento extra*

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeU_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeQ_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeK_Diabetes,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

> ### *Conclusiones*

Las conclusiones que hemos obtenido tras analizar los resultados de la evaluación de modelos son las siguientes:
* El mejor modelo que hemos obtenido ha sido el modelo del árbol de decisión sin discretización y con el preprocesamiento básico con una tasa de verdaderos positivos de 0.60.
* En general la tasa de veraderos positivos que hemos obtenidos de todos los modelos es bastante baja, siendo la más alta 0.6.
* La eliminación de outliers ha sobreajustado casi todos los modelos dando en la mayoría peores resultados para la tasa de verdaderos positivos, expecto para el modelo  de árbol de decisión con discretización basado en K medians cuya tasas de verdaderos positivos es 0.52 cuando se eliminan outliers y 0.43 sin la eliminación de outliers.
* En el caso de la discretización, lo más recomendable es no realizar una discretización a este conjunto de datos si usamos como modelo un árbol de decisión, sin embargo la discretización que mejores resultados ha dado sin eliminación de outliers es la de igual frecuencia (con una tasa de verdaderos positivos de 0.58) y con eliminación de outliers es la basada en K medians (con una tasa de verdaderos positivos de 0.52). No obstante; están ambas por debajo del modelo del árbol de decisión sin discretizar con o sin eliminación de outliers.


## *6.3 Titanic*

Teniendo en cuenta que la base de datos Titanic surge de un problema estadístico normal y corriente basado en el número de supervivientes de una tragedia, en este caso la tragedia del hundimiento del titanic, la métrica que utilizaremos para evaluar la calidad de los modelos generados podría ser o recall o precission ya que accuracy funciona mál con problemas desbalanceados y este lo está. Así que si tenemos que elegir alguna de las dos métricas, podemos optar por utilizar como métrica la puntuación F1 que combina las métricas de recall y preccision.

Finalmente, al utilizarse la base de datos para predecir si un viajero sobrevivió al Titanic dependiendo de ciertas caracterísitcas del viaje de este, consideraremos que dicha predicción es positiva si la variable objetivo Survived tiene el valor 1.

> ### *Modelos sin preprocesamiento*

In [ ]:
utils.EvaluationWClassRepo(zero_r_model,
               XTitanic_train, XTitanic_test,
               yTitanic_train, yTitanic_test)

Debido a que los datos originales que estamos manjeando poseen variables categóricas, no podemos evaluar el modelo árbol de decisión para nuestros datos sin aplcarle anteriormente un preprocesamiento. Por eso pasamos directamente a evaluar los modelos de árbol de decisión aplicándoles el preprocesamiento.

> ### *Modelos con el preprocesamiento desarrollado anteriormente*

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_Titanic,
               XTitanic_train, XTitanic_test,
               yTitanic_train, yTitanic_test)

In [ ]:

utils.EvaluationWClassRepo(preprocess_treemodel_discretizeU_Titanic,
               XTitanic_train, XTitanic_test,
               yTitanic_train, yTitanic_test)


In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeQ_Titanic,
               XTitanic_train, XTitanic_test,
               yTitanic_train, yTitanic_test)

In [ ]:
utils.EvaluationWClassRepo(preprocess_treemodel_discretizeK_Titanic,
               XTitanic_train, XTitanic_test,
               yTitanic_train, yTitanic_test)

> ### *Modelos con el preprocesamiento extra*

In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_Titanic,
               XTitanic_train, XTitanic_test,
               yTitanic_train, yTitanic_test)


In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeU_Titanic,
               XTitanic_train, XTitanic_test,
               yTitanic_train, yTitanic_test)


In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeQ_Titanic,
               XTitanic_train, XTitanic_test,
               yTitanic_train, yTitanic_test)


In [ ]:
utils.EvaluationWClassRepo(extra_preprocess_treemodel_discretizeK_Titanic,
               XTitanic_train, XTitanic_test,
               yTitanic_train, yTitanic_test)


> ### *Conclusiones*

Las conclusiones que hemos obtenido tras analizar los resultados de la evaluación de modelos son las siguientes:
* El mejor modelo que hemos obtenido ha sido el modelo del árbol de decisión con discretización con estrategia de igual anchura y con eliminación de outliers con un valor de f1 de 0.71.
* En general el valor f1 obtenido ha sido muy dispar, teniendo la mayoria de modelos resultados muy normales, pero estando presentes otros modelos con valores muy malos como el modelo de árbol de decisión con discretización en igual frecuencia y eliminación de outliers que ha tenido un valor de f1 de 0.38 mientras que el modelo con mejor valor ha sido el modelo del árbol de decisión con discretización con estregia de igual anchura y con eliminación de outliers con un valor muy bueno de f1 0.71.
* La eliminación de outliers ha sobreajustado casi todos los modelos dando en la mayoría peores resultados para el valor de f1, expecto para el modelo de árbol de decisión con discretización con estregia de igual anchura y con eliminación de outliers con un valor muy bueno de f1 0.71.
* En el caso de la discretización, lo más recomendable es realizar una discretización con estrategia basada en k-means si no realizamos eliminación de outliers pues hemos obtenido con ese modelo un f1 con valor 0.69, mientras que si realizamos outliers la mejor discretización es la de igual anchura pues ha obtenido un valor de f1 de 0.71.